# Practical Business Python: Efficiently Cleaning Text with Pandas
Chris Moffitt. "Efficiently Cleaning Text in Pandas. _Practical Business Python_, 16 Feb. 2021, https://pbpython.com/text-cleaning.html.

In [32]:
import pandas as pd
import numpy as np
import sidetable

Demonstrate with [open data set](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy) from Iowa. Large set, a 565 MB csv with 24 columns and 2.3 million rows. Using 2019 data. Download [here](https://data.iowa.gov/Sales-Distribution/Iowa-Liquor-Sales/m3tr-qhgy).

In [33]:
df = pd.read_csv('2019_Iowa_Liquor_Sales.csv')

In [3]:
df

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,INV-16681900011,01/02/2019,5286,Sauce,"108, College",Iowa City,52240.0,NaN,52.0,JOHNSON,...,48099,Hennessy VS,24,200,6.24,9.36,24,224.64,4.80,1.26
1,INV-16681900027,01/02/2019,5286,Sauce,"108, College",Iowa City,52240.0,NaN,52.0,JOHNSON,...,89191,Jose Cuervo Especial Reposado Mini,12,500,11.50,17.25,12,207.00,6.00,1.58
2,INV-16681900018,01/02/2019,5286,Sauce,"108, College",Iowa City,52240.0,NaN,52.0,JOHNSON,...,8824,Lauder's,24,375,3.21,4.82,24,115.68,9.00,2.37
3,INV-16685400036,01/02/2019,2524,Hy-Vee Food Store / Dubuque,3500 Dodge St,Dubuque,52001.0,NaN,31.0,DUBUQUE,...,35917,Five O'Clock Vodka,12,1000,4.17,6.26,12,75.12,12.00,3.17
4,INV-16690300035,01/02/2019,4449,Kum & Go #121 / Urbandale,12041 Douglas Pkwy,Urbandale,50322.0,NaN,77.0,POLK,...,36304,Hawkeye Vodka,24,375,1.86,2.79,24,66.96,9.00,2.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380340,INV-24281000038,12/31/2019,4557,Hometown Foods / Traer,420 Second St,Traer,50675.0,POINT (-92.468086 42.193386),86.0,TAMA,...,55106,Phillips Blackberry Brandy,12,750,4.76,7.14,4,28.56,3.00,0.79
2380341,INV-24284800010,12/31/2019,5514,Johncy's Liquor Store,585 Hwy 965 Ste D & E,North Liberty,52317.0,POINT (-91.60761 41.738129),52.0,JOHNSON,...,8828,Lauders,6,1750,11.18,16.77,6,100.62,10.50,2.77
2380342,INV-24274100040,12/31/2019,3932,Main Street Spirits / Mapleton,311 Main St,Mapleton,51034.0,POINT (-95.79375 42.165915),67.0,MONONA,...,80486,Tippy Cow Orange Cream,6,750,10.00,15.00,1,15.00,0.75,0.19
2380343,INV-24278900030,12/31/2019,5447,New Star / Raymond,101 Comerical Street,Raymond,50667.0,NaN,7.0,BLACK HAWK,...,18048,Evan Williams Green Label,6,1750,11.50,17.25,6,103.50,10.50,2.77


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380345 entries, 0 to 2380344
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               float64
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            int64  
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      float64
 19  State Bottle Retail    float64
 20  Bottles Sold           int64  
 21  Sale (Dollars)         float64
 22  Volume Sold (Liter

In [5]:
df.describe()

,Store Number,Zip Code,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
count,2.380345e+06,2.375581e+06,2.375581e+06,2.377427e+06,2.380344e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06,2.380345e+06
mean,3.903901e+03,5.126670e+04,5.730555e+01,1.052185e+06,2.647152e+02,4.836175e+04,1.242525e+01,8.766395e+02,1.034431e+01,1.551888e+01,1.127750e+01,1.467100e+02,9.368987e+00,2.469610e+00
std,1.138921e+03,9.881870e+02,2.727291e+01,9.329829e+04,1.370791e+02,6.708207e+04,8.108758e+00,5.214263e+02,8.568864e+00,1.285549e+01,3.131091e+01,4.871768e+02,3.824454e+01,1.010336e+01
min,2.106000e+03,5.000200e+04,1.000000e+00,1.011100e+06,3.300000e+01,1.590000e+02,1.000000e+00,2.000000e+01,8.900000e-01,1.340000e+00,1.000000e+00,1.340000e+00,2.000000e-02,0.000000e+00
25%,2.624000e+03,5.031600e+04,3.100000e+01,1.012200e+06,1.150000e+02,2.682800e+04,6.000000e+00,7.500000e+02,5.500000e+00,8.250000e+00,3.000000e+00,3.375000e+01,1.500000e+00,3.900000e-01
50%,3.952000e+03,5.110300e+04,6.200000e+01,1.031200e+06,2.600000e+02,3.817700e+04,1.200000e+01,7.500000e+02,8.250000e+00,1.238000e+01,6.000000e+00,7.536000e+01,4.800000e+00,1.260000e+00
75%,4.971000e+03,5.230200e+04,7.700000e+01,1.062400e+06,3.890000e+02,6.486400e+04,1.200000e+01,1.000000e+03,1.296000e+01,1.944000e+01,1.200000e+01,1.485600e+02,1.050000e+01,2.770000e+00
max,9.042000e+03,5.722200e+04,9.900000e+01,1.901200e+06,9.780000e+02,9.992920e+05,4.800000e+01,6.000000e+03,1.749120e+03,2.623680e+03,6.750000e+03,7.843500e+04,1.181250e+04,3.120530e+03


Use [sidetable](https://pypi.org/project/sidetable/) to summarize data.

In [6]:
df.stb.freq(['Store Name'], value='Sale (Dollars)', style=True, cum_cols=False)

,Store Name,Sale (Dollars),percent
0,Central City 2,"11,877,164",3.40%
1,Hy-Vee #3 / BDI / Des Moines,"11,275,152",3.23%
2,Hy-Vee Wine and Spirits / Iowa City,"5,001,156",1.43%
3,Wilkie Liquors,"3,639,515",1.04%
4,Lot-A-Spirits,"3,504,665",1.00%
5,Costco Wholesale #788 / WDM,"3,178,079",0.91%
6,Sam's Club 8162 / Cedar Rapids,"3,147,579",0.90%
7,Benz Distributing,"3,082,936",0.88%
8,Hy-Vee Food Store / Urbandale,"3,073,798",0.88%
9,Sam's Club 6344 / Windsor Heights,"2,963,108",0.85%


## Cleaning attempt no. 1
Use .loc with a boolean filter (.str accessor) on each string in the 'Store Name' column.

First with and then without regex turned off.

In [7]:
%%timeit
df.loc[df['Store Name'].str.contains('Hy-Vee', case=False), 'Store_Group_1'] = 'Hy-Vee'

1.14 s ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
df.loc[df['Store Name'].str.contains('Hy-Vee', case=False, regex=False), 'Store_Group_1'] = 'Hy-Vee'

684 ms ± 13.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
df['Store_Group_1'].value_counts(dropna=False)

NaN       1617777
Hy-Vee     762568
Name: Store_Group_1, dtype: int64

## Cleaning attempt no. 2
Use np.select to run multiple searches and apply a specified value on match. Resources: [Article](https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/) on Dataquest, [presentation](https://docs.google.com/presentation/d/1X7CheRfv0n4_I21z4bivvsHt6IDxkuaiAuCclSzia1E/edit#slide=id.g635adc05c1_1_1840) by Nathan Cheever.

In [10]:
store_patterns = [
    (df['Store Name'].str.contains('Hy-Vee', case=False, regex=False), 'Hy-Vee'),
    (df['Store Name'].str.contains('Central City',
                                case=False,  regex=False), 'Central City'),
    (df['Store Name'].str.contains("Smokin' Joe's",
                                case=False,  regex=False), "Smokin' Joe's"),
    (df['Store Name'].str.contains('Walmart|Wal-Mart',
                                case=False), 'Wal-Mart'),
    (df['Store Name'].str.contains('Fareway Stores',
                                case=False,  regex=False), 'Fareway Stores'),
    (df['Store Name'].str.contains("Casey's",
                                case=False,  regex=False), "Casey's General Store"),
    (df['Store Name'].str.contains("Sam's Club", case=False,  regex=False), "Sam's Club"),
    (df['Store Name'].str.contains('Kum & Go',  regex=False, case=False), 'Kum & Go'),
    (df['Store Name'].str.contains('CVS',  regex=False, case=False), 'CVS Pharmacy'),
    (df['Store Name'].str.contains('Walgreens',  regex=False, case=False), 'Walgreens'),
    (df['Store Name'].str.contains('Yesway',  regex=False, case=False), 'Yesway Store'),
    (df['Store Name'].str.contains('Target Store',  regex=False, case=False), 'Target'),
    (df['Store Name'].str.contains('Quik Trip',  regex=False, case=False), 'Quik Trip'),
    (df['Store Name'].str.contains('Circle K',  regex=False, case=False), 'Circle K'),
    (df['Store Name'].str.contains('Hometown Foods',  regex=False,
                                case=False), 'Hometown Foods'),
    (df['Store Name'].str.contains("Bucky's", case=False,  regex=False), "Bucky's Express"),
    (df['Store Name'].str.contains('Kwik', case=False,  regex=False), 'Kwik Shop')
]

Easy to get conditions and values mismatched when using np.select, so here combine them into a tuple to help track data matches. List of tuples needs to be broken into two separate lists using zip.

In [11]:
store_criteria, store_values = zip(*store_patterns)
df['Store_Group_1'] = np.select(store_criteria, store_values, 'other')

Once again use sidetable to summarize:

In [12]:
df.stb.freq(['Store_Group_1'], value='Sale (Dollars)', style=True, cum_cols=False)

,Store_Group_1,Sale (Dollars),percent
0,Hy-Vee,"126,265,195",36.16%
1,other,"112,733,367",32.28%
2,Fareway Stores,"23,146,939",6.63%
3,Wal-Mart,"22,641,682",6.48%
4,Sam's Club,"19,604,085",5.61%
5,Central City,"14,108,944",4.04%
6,Casey's General Store,"11,351,935",3.25%
7,Kum & Go,"6,019,449",1.72%
8,Walgreens,"2,942,270",0.84%
9,Target,"2,904,611",0.83%


Still almost a third of revenue in "other" accounts. Maybe if an account doesn't match we can use 'Store Name' instead of lumping together with 'other'. The following uses combine_first to fill in the 'None' values with 'Store Name':

In [28]:
%%timeit
df['Store_Group_1'] = np.select(store_criteria, store_values, None)
df['Store_Group_1'] = df['Store_Group_1'].combine_first(df['Store Name'])

419 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Now look at the data:

In [14]:
df.stb.freq(['Store_Group_1'], value='Sale (Dollars)', style=True, cum_cols=False)

,Store_Group_1,Sale (Dollars),percent
0,Hy-Vee,"126,265,195",36.16%
1,Fareway Stores,"23,146,939",6.63%
2,Wal-Mart,"22,641,682",6.48%
3,Sam's Club,"19,604,085",5.61%
4,Central City,"14,108,944",4.04%
5,Casey's General Store,"11,351,935",3.25%
6,Kum & Go,"6,019,449",1.72%
7,Wilkie Liquors,"3,639,515",1.04%
8,Lot-A-Spirits,"3,504,665",1.00%
9,Costco Wholesale #788 / WDM,"3,178,079",0.91%


## Cleaning attempt no. 3

Matt Harrison's generalize function.

In [34]:
def generalize(ser, match_name, default=None, regex=False, case=False):
    """ Search a series for text matches.
    Based on code from https://www.metasnake.com/blog/pydata-assign.html

    ser: pandas series to search
    match_name: tuple containing text to search for and text to use for normalization
    default: If no match, use this to provide a default value, otherwise use the original text
    regex: Boolean to indicate if match_name contains a  regular expression
    case: Case sensitive search

    Returns a pandas series with the matched value

    """
    seen = None
    for match, name in match_name:
        mask = ser.str.contains(match, case=case, regex=regex)
        if seen is None:
            seen = mask
        else:
            seen |= mask
        ser = ser.where(~mask, name)
    if default:
        ser = ser.where(seen, default)
    else:
        ser = ser.where(seen, ser.values)
    return ser

New pattern list to apply:

In [35]:
store_patterns_2 = [('Hy-Vee', 'Hy-Vee'), ("Smokin' Joe's", "Smokin' Joe's"),
                    ('Central City', 'Central City'),
                    ('Costco Wholesale', 'Costco Wholesale'),
                    ('Walmart', 'Walmart'), ('Wal-Mart', 'Walmart'),
                    ('Fareway Stores', 'Fareway Stores'),
                    ("Casey's", "Casey's General Store"),
                    ("Sam's Club", "Sam's Club"), ('Kum & Go', 'Kum & Go'),
                    ('CVS', 'CVS Pharmacy'), ('Walgreens', 'Walgreens'),
                    ('Yesway', 'Yesway Store'), ('Target Store', 'Target'),
                    ('Quik Trip', 'Quik Trip'), ('Circle K', 'Circle K'),
                    ('Hometown Foods', 'Hometown Foods'),
                    ("Bucky's", "Bucky's Express"), ('Kwik', 'Kwik Shop')]

Call out the data:

In [17]:
df['Store_Group_2'] = generalize(df['Store Name'], store_patterns_2)

In [18]:
%%timeit
df['Store_Group_2'] = generalize(df['Store Name'], store_patterns_2)

11.9 s ± 76.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


That took a crap-ton longer. I don't care how "elegant" it is, that's a hefty price to pay!

In [19]:
df.stb.freq(['Store_Group_2'], value='Sale (Dollars)', style=True, cum_cols=False)

,Store_Group_2,Sale (Dollars),percent
0,Hy-Vee,"126,265,195",36.16%
1,Fareway Stores,"23,146,939",6.63%
2,Walmart,"22,641,682",6.48%
3,Sam's Club,"19,604,085",5.61%
4,Central City,"14,108,944",4.04%
5,Casey's General Store,"11,351,935",3.25%
6,Kum & Go,"6,019,449",1.72%
7,Costco Wholesale,"5,777,393",1.65%
8,Wilkie Liquors,"3,639,515",1.04%
9,Lot-A-Spirits,"3,504,665",1.00%


## What about data types?

Converting strings to string data type doesn't seem to make a difference, but converting to a category time using astype can:

In [23]:
df['Store Name'] = df['Store Name'].astype('category')

In [27]:
%%timeit
store_criteria, store_values = zip(*store_patterns)
df['Store_Group_3'] = np.select(store_criteria, store_values, None)
df['Store_Group_3'] = df['Store_Group_1'].combine_first(df['Store Name'])

387 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)




## Lookup table
Data set has a lot of duplicated data. Build a lookup table an process resource intensive function only once per string.

Convert back to string from category:


In [36]:
df['Store Name'] = df['Store Name'].astype('string')

Now build lookup table:

In [37]:
lookup_df = pd.DataFrame()
lookup_df['Store Name'] = df['Store Name'].unique()
lookup_df['Store_Group_5'] = generalize(lookup_df['Store Name'], store_patterns_2)

In [38]:
lookup_df

,Store Name,Store_Group_5
0,Sauce,Sauce
1,Hy-Vee Food Store / Dubuque,Hy-Vee
2,Kum & Go #121 / Urbandale,Kum & Go
3,IDA Liquor,IDA Liquor
4,Lake View Foods,Lake View Foods
...,...,...
1754,Hometown Foods - Conrad,Hometown Foods
1755,Casey's General Store #72 / Tipton,Casey's General Store
1756,Hometown Foods - Hubbard,Hometown Foods
1757,Shortee's Pit Stop / Speedway Cafe,Shortee's Pit Stop / Speedway Cafe


In [40]:
df = pd.merge(df, lookup_df, how='left')

For some reason, above blows up when I try to use %%timeit.

## Summary
Importance of data cleaning in Randy Au's [newsletter](https://counting.substack.com/p/data-cleaning-is-analysis-not-grunt).

In summary:

|Solution                  |Execution time|Notes                    |
|:-------------------------|:-------------|:------------------------|
|np.select                 |13s           |Can work for non-text analysis|
|generalize                |15s           |Text only|
|Category Data and np.select|786ms        |Categorical data can get tricky when merging and joining|
|Lookup table and generalize|1.3s         |A lookup table can be maintained by someone else|
